# Create BRISE dataset for POTATO

In [2]:
from potato.dataset.dataset import Dataset
from potato.models.trainer import GraphTrainer

In [3]:
TRAIN_DIR = "../../../../data/train"

In [4]:
import os
import json

GOLD_ATTRIBUTES = "gold_attributes"
GOLD = "labels_gold"
ANNOTATED_ATTRIBUTES = "annotated_attributes"

def load_data(dir_path, only_gold=True):
    data = []
    for filename in os.listdir(dir_path):
        with open(os.path.join(dir_path, filename), "rt") as f:
            doc = json.load(f)
            if only_gold and not doc[GOLD]:
                continue
            for sen in doc["sens"].values():
                if doc[GOLD]:
                    labels = list(set(sen[GOLD_ATTRIBUTES].keys()))
                else:
                    labels = list(set(sen[ANNOTATED_ATTRIBUTES].keys()))
                data.append({
                    "sen_id": sen["id"],
                    "text": sen["text"],
                    "labels": labels,
                })
    return data

In [5]:
NOT = "NOT"

def get_sentences(data, attribute):
    sentences = []
    for sen in data:
        if attribute in sen["labels"]:
            sentences.append((sen["text"], attribute))
        else:
            sentences.append((sen["text"], NOT))       
    return sentences

In [8]:
data = load_data(TRAIN_DIR, only_gold=True)

In [9]:
len(data)

1154

In [10]:
data[10:15]

[{'sen_id': '8025_7_0',
  'text': 'Bestimmungen ohne Bezeichnung des Geltungsbereichs mit dem Planzeichen BB:',
  'labels': ['Planzeichen']},
 {'sen_id': '8025_8_0',
  'text': 'Für das gesamte Plangebiet wird bestimmt: Dächer dürfen die festgesetzte Gebäudehöhe um höchstens 4,5 m überragen.',
  'labels': ['GebaeudeHoeheArt',
   'PlangebietAllgemein',
   'AbschlussDachMaxBezugGebaeude']},
 {'sen_id': '8025_9_0',
  'text': 'Im Bauland/Wohngebiet darf die Bruttogeschoßfläche aller Geschoße, die ganz oder teilweise über dem anschließenden Gelände liegen, insgesamt höchstens 11.500 m² betragen.',
  'labels': ['WidmungUndZweckbestimmung', 'Flaechen']},
 {'sen_id': '8025_10_0',
  'text': 'Bestimmungen mit Bezeichnung des Geltungsbereichs mit dem Planzeichen BB:',
  'labels': ['Planzeichen']},
 {'sen_id': '8025_11_0',
  'text': 'Für die mit BB1 bezeichneten Grundflächen wird bestimmt: Die Unterbrechung der geschlossenen Bauweise ist zulässig',
  'labels': ['Planzeichen', 'UnterbrechungGeschlos

## Planzeichen

In [11]:
sentences = get_sentences(data, "Planzeichen")

In [12]:
sentences[10:15]

[('Bestimmungen ohne Bezeichnung des Geltungsbereichs mit dem Planzeichen BB:',
  'Planzeichen'),
 ('Für das gesamte Plangebiet wird bestimmt: Dächer dürfen die festgesetzte Gebäudehöhe um höchstens 4,5 m überragen.',
  'NOT'),
 ('Im Bauland/Wohngebiet darf die Bruttogeschoßfläche aller Geschoße, die ganz oder teilweise über dem anschließenden Gelände liegen, insgesamt höchstens 11.500 m² betragen.',
  'NOT'),
 ('Bestimmungen mit Bezeichnung des Geltungsbereichs mit dem Planzeichen BB:',
  'Planzeichen'),
 ('Für die mit BB1 bezeichneten Grundflächen wird bestimmt: Die Unterbrechung der geschlossenen Bauweise ist zulässig',
  'Planzeichen')]

### UD

In [13]:
dataset = Dataset(sentences, label_vocab={"NOT":0, "Planzeichen": 1})

In [15]:
df = dataset.to_dataframe()
df

,text,label,label_id,graph
0,MAGISTRAT DER STADT WIEN MA 21 Stadtteilplanun...,NOT,0,None
1,Der Gemeinderat hat in seiner Sitzung am 29. A...,NOT,0,None
2,"Bezirk, Kat. Großjedlersdorf I werden unter An...",NOT,0,None
3,Die bisherigen Flächenwidmungspläne und Bebauu...,NOT,0,None
4,Gemäß § 4 und § 5 der BO für Wien sowie § 48 d...,NOT,0,None
...,...,...,...,...
1149,Auf den mit G BB2 bezeichneten Flächen ist die...,Planzeichen,1,None
1150,Innerhalb der mit BB 3 bezeichneten Flächen si...,Planzeichen,1,None
1151,Die Ausgestaltung hat zumindest durch das Aufs...,NOT,0,None
1152,Für die mit BB 4 bezeichneten Bereiche ist die...,Planzeichen,1,None


In [16]:
dataset.set_graphs(dataset.parse_graphs(graph_format="ud"))

2021-11-18 10:48:33 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-11-18 10:48:33 INFO: Use device: cpu
2021-11-18 10:48:33 INFO: Loading: tokenize
2021-11-18 10:48:33 INFO: Loading: pos
2021-11-18 10:48:33 INFO: Loading: lemma
2021-11-18 10:48:34 INFO: Loading: depparse
2021-11-18 10:48:35 INFO: Loading: sentiment
2021-11-18 10:48:36 INFO: Loading: ner
2021-11-18 10:48:36 INFO: Done loading processors!
100%|█████████████████████████████████████████████████████████████| 1154/1154 [05:44<00:00,  3.35it/s]


In [17]:
df = dataset.to_dataframe()

In [19]:
import pickle

with open("planzeichen_gold_ud.pickle", "wb") as f:
    pickle.dump(df.graph, f)